# Welcome to the FunC interactive cheat sheet!
This tutorial is still in work, any contribution is highly appreciated!  
Language basics will be covered, more information can be found in the TON repo:
1. [TVM docs](https://test.ton.org/tvm.pdf)
2. [Tests](https://github.com/ton-blockchain/ton/tree/9c9248a9ae1791152d88c9c36574b4becdf5e8b6/crypto/func/test)
3. [Sample contracts](https://github.com/ton-blockchain/ton/tree/9c9248a9ae1791152d88c9c36574b4becdf5e8b6/crypto/smartcont)

Basically, FunC is quite lightweight abstraction on top of TVM assembler, so if you read TVM spec you will likely get the core concepts very quickly. 

## 🍬 Xeus-Fift specific features
### Include macro

In [93]:
#include "stdlib.fc"  ;; searches for file in the FUNCPATH and workdir

### Main-wrapper


### Force return

## Integer operations
Underlying TVM type: signed 257-bit integers, representing integer numbers in the
range -2^256 ... 2^256 − 1

* Arithmetic `+`  `-`  `*`  `/`  `%`   
* Bitwise `&`  `|`  `<<`  `>>`   
* Boolean `<`  `<=`  `>`  `>=`  `==`  `!=` 

In [59]:
0x01 ^ 0x80  ;; xor

129

In [60]:
8 <=> -4  ;; compare (1 if equal, -1 if less, 0 otherwise)

1

In [210]:
~ 0xff + 0xff  ;; bitwise complement

-1

In [214]:
min(-2, 0)

-2

In [216]:
max(2, 3)

3

### Division rounding rules

In [38]:
9 ~>> 2  ;; division by 2^n using nearest-integer rounding

2

In [39]:
9 ^>> 2  ;; division by 2^n using ceiling rounding

3

Rounding works similarly for integer division and modular division operators:  
`~/`  `^/`  `~%`  `^%`

### Compound operators

In [40]:
11 /% 2  ;; extended mod division: computes both the quotient and the remainder

5 1

In [65]:
divmod(11, 2)  ;; same effect

5 1

In [66]:
moddiv(11, 2)  ;; change output order

1 5

In [67]:
int x = 11;  ;; declaring variable of type `int`
x~divmod(2)  ;; non-const method (starting with `~`)

5 1

In [98]:
int x = 42;
x += 2

44

Similar let-constructions can be made on top of with practically all integer operators:  
`-=` `*=` `/=` `%=`  
`&=` `|=` `<<=` `>>=` `^=`  
`~>>=` `^>>=` `~/=` `^/=` `~%=` `^%=`

In [104]:
muldiv(3, 3, 2)  ;; equivalent of `a * b / c`

4

In [105]:
muldivr(3, 3, 2)  ;; equivalent of `a * b ~/ c`

5

### Constants

In [87]:
true

-1

In [191]:
false

0

In [192]:
~ false

-1

## Tuple manipulation
### FunC syntax convenience

In [194]:
var t = (1, 2, 3);  ;; packing
t

1 2 3

In [174]:
var (a, b, c) = (1, 2, 3);  ;; unpacking
b

2

In [175]:
var (a, b) = (1, (2, 3));
b

2 3

In [234]:
var (_, x, _) = (0xab, 0xcd, 0xef);  ;; one can use special operand `_` for values that won't be used
x

205

### TVM tuples
An ordered collection of up to 255 components, having arbitrary value types, possibly distinct.  
May be used to represent nonpersistent values of arbitrary algebraic data types.

In [176]:
Nil  ;; 0-tuple

[]

#### Pairs

In [177]:
pair(0xdead, 0xbeaf)

[ 57005 48815 ]

In [208]:
second(pair(0xdead, 0xbeaf))

48815

In [217]:
at(pair(0xdead, 0xbeaf), 0)  ;; universal helper for tuples

57005

In [179]:
unpair(pair(0xdead, 0xbeaf))

57005 48815

#### Triples

In [180]:
triple(1, 2, 3)

[ 1 2 3 ]

In [202]:
third(triple(1, 2, 3))

3

In [182]:
untriple(triple(1, 2, 3))

1 2 3

#### 4-tuples

In [183]:
tuple4(-10, -20, -30, -40)

[ -10 -20 -30 -40 ]

In [184]:
fourth(tuple4(-10, -20, -30, -40))

-40

In [185]:
untuple4(tuple4(-10, -20, -30, -40))

-10 -20 -30 -40

#### Lists

In [186]:
cons(1, cons(2, cons(3, Nil)))

[ 1 [ 2 [ 3 [] ] ] ]

In [187]:
uncons(cons(1, cons(2, cons(3, Nil))))  ;; (head, tail)

1 [ 2 [ 3 [] ] ]

In [188]:
car(cons(1, cons(2, cons(3, Nil))))  ;; head

1

In [189]:
cdr(cons(1, cons(2, cons(3, Nil))))  ;; tail

[ 2 [ 3 [] ] ]

In [190]:
list_next(cons(1, cons(2, cons(3, Nil))));  ;; (tail, head)

[ 2 [ 3 [] ] ] 1

## Cell manipulation
A TVM cell consists of at most 1023 bits of data, and of at
most four references to other cells.  
All persistent data (including TVM
code) in the TON Blockchain is represented as a collection of TVM
cells.  

The cell primitives are mostly either cell serialization primitives, which work
with Builder s, or cell deserialization primitives, which work with Slices.

### Cell serialization primitives
#### Builder
A TVM cell builder, or builder for short, is an “incomplete”
cell that supports fast operations of appending bitstrings and cell references at its end.  
Builders are used for packing (or serializing) data
from the top of the stack into new cells (e.g., before transferring them
to persistent storage).

In [255]:
builder b = begin_cell()  ;; two descriptor bytes come first (read more in 3.1.4. Standard cell representation)

BC{0000}

In [246]:
cell res = end_cell(begin_cell());

C{96A296D224F285C67BEE93C30F8A309157F0DAA35DC5B87E410B78630A09CFC7}

In [302]:
begin_cell().end_cell()  ;; const method (dot is not a part of the function name)

C{96A296D224F285C67BEE93C30F8A309157F0DAA35DC5B87E410B78630A09CFC7}

In [281]:
begin_cell().store_int(-0xFF, 32)  ;; we added a 32bit signed integer

BC{0008ffffff01}

In [283]:
begin_cell().store_uint(-42, 32)

VM error: integer out of range

In [284]:
cell c1 = begin_cell().store_int(1, 8).end_cell();
begin_cell().store_ref(c1)  ;; first byte indicates that cell holds one reference

BC{0100}

#### Slice
A TVM cell slice, or slice for short, is a contiguous “sub-cell”
of an existing cell, containing some of its bits of data and some of its
references.  
Essentially, a slice is a read-only view for a subcell of a cell.  
Slices are used for unpacking data previously stored (or serialized) in a
cell or a tree of cells.

In [307]:
cell c = begin_cell().store_int(0xfa, 16).end_cell();
c.begin_parse()

CS{Cell{000400fa} bits: 0..16; refs: 0..0}

In [315]:
cell c = begin_cell().store_int(0xfa, 16).end_cell();
slice cs = c.begin_parse();
int res = cs~load_int(16);
res

250

In [334]:
cell c = begin_cell().store_uint(0xfa, 16).end_cell();
slice cs = c.begin_parse();
var (b1, b2) = (cs~load_uint(16), cs~load_uint(16))  ;; non-const method `load` reads and increases offset

VM error: cell underflow

In [430]:
cell c = begin_cell().store_int(0xfa, 16).end_cell();
slice cs = c.begin_parse();
cs.skip_bits(16).end_parse()  ;; ensure we read the entire cell with `end_parse`

In [397]:
cell c = begin_cell().store_int(0xfa, 9).end_cell();
slice cs = c.begin_parse();
slice c8 = cs.skip_bits(1).first_bits(8);
c8.preload_uint(8)  ;; read without increasing the offset

250

In [416]:
cell c1 = begin_cell().store_uint(1, 1).end_cell();
cell c2 = begin_cell().store_ref(c1).end_cell();
cell c3 = c2.begin_parse().preload_ref();
c3.begin_parse().preload_uint(1)

1

#### Hashmaps
Hashmaps, or dictionaries, are a specific data structure represented by a tree
of cells.

In [294]:
new_dict()

(null)

In [295]:
dict_empty?(new_dict())

-1

In [417]:
var d = new_dict();
var b = begin_cell().store_int(0xff, 32);
d~idict_set_builder(32, 0xff, b);
d.begin_parse()

CS{Cell{0012a0000000ff000000ff} bits: 0..72; refs: 0..0}

In [418]:
;; TODO:

## Subroutines
### Return type
Can be any combination (nested funC tuples) of builtin types `int` `tuple` `cell` `slice` `builder` `cont`, special "hole" type `_` (type is deduced from the return value), or void (empty tuple) `()`

### Modifiers
* `impure` - tells the compiler that function can modify passed arguments, and prevents function invocation pruning during the optimization;
* `inline` - works pretty the same as in C, tells the compiler to paste function body on every call occurence;
* `inline_ref` - 
* `method_id` - declares a Get method (entrypoint) for the contract (that can be called along with the `main`, `recv_internal` and `recv_external`)

### ASM bindings
`asm` ( `stack order of arguments` -> `stack order of return values` ) "`INSTRUCTION#0`" "`INSTRUCTION#1`" ... "`INSTRUCTION#N`"
Here is a comprehensive sample function from stdlib demonstrating all possible outcomes:

In [426]:
(int, slice, int) udict_get_next?(cell dict, int key_len, int pivot) asm(pivot dict key_len -> 1 0 2) "DICTUGETNEXT" "NULLSWAPIFNOT" "NULLSWAPIFNOT";

One can omit both or separate args/return stack ordering:

In [427]:
int dict_empty?(cell c) asm "DICTEMPTY";

In [428]:
(slice, cell) load_ref(slice s) asm( -> 1 0) "LDREF";

In [429]:
cell idict_set_ref(cell dict, int key_len, int index, cell value) asm(value index dict key_len) "DICTISETREF";

### "Generics"
`forall` `X0`, `X1`, ... , `Xn` -> 

In [425]:
forall X, Y -> tuple pair(X x, Y y) asm "PAIR";

## Conditional and loop statements

### Ternary operator

In [435]:
0 == 0 ? 1 : 2

## Exceptions